## 맞춤형 YouTube 컨텐츠 추출
#### 1. 환경 변수 Load

In [1]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
#   OPENAI_API_KEY = "" # OpenAI 구독 후 획득
#   LANGCHAIN_TRACING_V2 = "true"
#   LANGCHAIN_ENDPOINT = "https://api.smith.langchain.com"
#   LANGCHAIN_API_KEY = "" # LangSmith 가입 후 획득
#   YOUTUBE_DEVELOPER_KEY="" # developers.google.com 가입 후 YouTube Data API enable
load_dotenv(dotenv_path='./../.env')

import os
os.environ["LANGCHAIN_PROJECT"] = "langchain_study"

#### 2. llm 객체(ChatGPT) 생성

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.3,
    model="gpt-4o",
)

#### 3. 사용자 요청에서 YouTube용 query와 LLM prompt를 추출(첫번째 Chain 만들기)
- 출력할 Schema 에 맞춰 Pydantic 객체 정의

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field

class YoutubeQuery(BaseModel):
    youtube_query: str = Field(description="youtube에서 사용할 검색어")
    llm_query: str = Field(description="YouTube 컨텐츠가 담고있는 Text 정보에서 사용자가 원하는 부분을 찾기 위한 자연어 질의")
    language: str = Field(description="검색할 컨텐츠의 한글, 영어 구분(ko, en 둘중에 하나만 가능)")

- PydanticOutputParser 생성

In [4]:
from langchain.output_parsers import PydanticOutputParser

parser_query = PydanticOutputParser(pydantic_object=YoutubeQuery)

- Prompt 생성

In [5]:
youtube_search_query_tips ="""
YouTube 검색 시 query 작성에 대한 주요 규칙과 팁:

1. 기본 검색:
   - 단순히 키워드나 문구를 입력합니다.
   - 예: `Python tutorial`

2. 정확한 구문 검색:
   - 큰따옴표로 묶어 정확한 구문을 검색합니다.
   - 예: `"Python for beginners"`

3. 불리언 연산자:
   - AND: 기본적으로 모든 단어는 AND로 연결됩니다.
   - OR: 단어 사이에 OR을 사용합니다.
   - 예: `Python OR Java tutorial`

4. 제외 검색:
   - 빼고 싶은 단어 앞에 마이너스(-) 기호를 사용합니다.
   - 예: `Python tutorial -advanced`

5. 와일드카드:
   - 별표(*)를 사용해 부분 일치를 검색할 수 있습니다.
   - 예: `Python * tutorial`

6. 채널 검색:
   - 채널명 앞에 @를 붙입니다.
   - 예: `@GoogleDevelopers Python`

7. 필터 사용:
   - 특정 필터를 쿼리에 직접 포함할 수 있습니다.
   - 예: `Python tutorial after:2023-01-01` (2023년 이후 영상)

8. 대소문자 구분:
   - YouTube 검색은 대소문자를 구분하지 않습니다.

9. 특수 문자:
   - 대부분의 특수 문자는 무시됩니다.

10. 길이 제한:
    - 쿼리 문자열에는 길이 제한이 있으므로 너무 길지 않게 작성합니다.

11. 언어와 지역:
    - 검색 결과는 사용자의 위치와 언어 설정에 영향을 받을 수 있습니다.

12. 동의어:
    - YouTube는 때때로 동의어를 자동으로 포함시킵니다.

이러한 규칙을 조합하여 더 정확하고 효과적인 검색 쿼리를 만들 수 있습니다. 예를 들어:

```python
query = '"Python for beginners" -advanced after:2023-01-01'
```

이 쿼리는 "Python for beginners"라는 정확한 구문을 포함하고, "advanced"를 제외하며, 2023년 1월 1일 이후에 업로드된 비디오를 검색합니다.
"""

In [6]:
from langchain.prompts import PromptTemplate

prompt_query = PromptTemplate.from_template(
"""
System :
너는 사용자가 원하는 youtube 컨텐츠를 찾아주는 AI야.

사용자의 query를 분석해서 다음 3가지 정보를 생성해줘.
 - 사용자가 원하는 YouTube 컨텐츠를 찾을 수 있도록 YouTube 검색어를 만들어줘.
 - 찾아야 할 컨텐츠가 한글인 경우 'ko' 영어인 경우 'en'이라고 표시해줘.
 - YouTube 컨텐츠가 담고있는 Text 정보에서 사용자가 원하는 부분을 찾기 위한 자연어 질의를 만들어줘
   예를 들어, 채널명이나 필터 정보 등 YouTube 검색에만 사용되는 내용은 제외해야되.
   
검색어를 만들때 tips 를 참고해.

tips :
{tips}

query : 
{query}

Format :
{format}
"""
)

# format에 PydanticOutputParser의 format 추가
prompt_query = prompt_query.partial(
    format = parser_query.get_format_instructions(),
    tips = youtube_search_query_tips,
    )

- LCEL Chain 생성

In [7]:
chain_1st = prompt_query | llm | parser_query

In [ ]:
chain_1st.output_schema.schema()

#### [TEST_01]. 첫번째 chain TEST

In [ ]:
# Test
long_query = """
Python 강의 중 동적 타이핑(Dynamic Typing)에 대한 부분만 찾아줘
"""
response_1st = chain_1st.invoke({"query":long_query})
response_1st

#### 4. YouTube에서 영상 Meta 정보와 자막을 조회하는 함수정의
- input : `language`,`youtube_query`

In [8]:
from youtube import ContentManager, Content, YouTube
from functions import get_youtube_captions

def results_from_youtube(response_1st):

    print(response_1st)
    language = response_1st.language
    youtube_query = response_1st.youtube_query
    llm_query = response_1st.llm_query

    yt = YouTube()

    articles = yt.search(
        query=youtube_query, 
        caption=True, 
        caption_language=language,
        #max_results = 10,
        max_results = 3,
    )

    c_manager = ContentManager()

    for a in articles:
        from pprint import pprint
        pprint(a)
        c_manager.add(Content.from_dict(a))

    for c in c_manager:
        c_manager.add_captions(c.id, get_youtube_captions(c.id, language=language))

    youtube_captions = [   
        {
            "query": llm_query,
            "captions": c_manager[content.id].captions,
            "video_id": content.id,
            "channel_name": c_manager[content.id].channel,
        } for content in c_manager
    ]

    return youtube_captions

#### [TEST_02]. YouTube에서 자막 가져오기

In [ ]:
# Test
youtube_captions = results_from_youtube(response_1st)
youtube_captions

#### 5. Youtube 자막을 분석해서 사용자 요청 내용 추출(두번째 Chain 만들기)
- 출력할 Schema 에 맞춰 Pydantic 객체 정의

In [9]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

class Segment(BaseModel):
    title: str = Field(description="해당 구간의 제목 (20자 이하)")
    start: float = Field(description="해당 구간의 시작 시간(초)")
    duration: float = Field(description="해당 구간의 duration (세부 구간들의 duration 합계)(초))")

class Summary(BaseModel):
    video_id: str = Field(description="동영상을 식별하는 id")
    channel_name: str = Field(description="채널 이름")
    description: str = Field(description="captions 내용 요약")
    segments: List[Segment] = Field(description="사용자 query를 만족하는 구간들 정보")

- PydanticOutputParser 생성

In [10]:
from langchain.output_parsers import PydanticOutputParser

parser_selection = PydanticOutputParser(pydantic_object=Summary)

- Prompt 생성

In [11]:
from langchain.prompts import PromptTemplate

prompt_selection = PromptTemplate.from_template(
"""
System :
너는 동영상 편집자이다.
먼저 captions 내용을 100자 이내로 요약해.
요약 결과는 네가 시청자에게 동영상을 소개하는 말투여야 해.
그리고 captions에서 사용자 query를 잘 설명하고 있는 내용을 뽑아내고
그 내용이 포함된 구간을 찾아내.
여기서 구간이란 captions 내의 하나의 세부 구간이 아니라 연속되는 여러개의 세부 구간의 합을 의미한다.

예) 

세부 구간 1 :
  start : 112.0, duration : 5.0
세부 구간 2 :
  start : 117.0, duration : 4.0
세부 구간 3 :
  start : 121.0, duration : 6.0
세부 구간 4 : 
  start : 127.0, duration : 3.0

네가 찾은 하나의 구간이 세부 구간 1 ~ 4 인 경우
구간의 start, duration은 다음과 같이 계산 하면 된다.
  start : 112.0 (세부 구간 1 의 start)
  duration : 18.0 ( 5.0 + 4.0 + 6.0 + 3.0, 세부 구간 4개 duration 의 합계)

해당 구간의 내용을 20자 이내로 요약해서 구간 제목을 만들어.
그 구간은 없을 수도 있고 여러개 일수도 있다.
그 구간이 4개 이상이라면 가장 적합한 것 3개만 찾아내라.
video_id 값은 동영상을 식별하는 id이다. 변경하지 말고 그대로 출력하라.
channel_name은 동영상을 소유한 채널 이름이다. 변경하지 말고 그대로 출력하라.

query : 
{query}

video_id :
{video_id}

channel_name :
{channel_name}

captions : 
{captions}

Format :
{format}
"""
)

# format에 PydanticOutputParser의 format 추가
prompt_selection = prompt_selection.partial(format=parser_selection.get_format_instructions())

- LCEL Chain 생성

In [12]:
chain_2nd = prompt_selection | llm | parser_selection

In [ ]:
chain_2nd.output_schema.schema()

#### [TEST_03]. YouTube 자막에서 Query에 적합한 부분 골라내기

In [ ]:
# youtube_captions : TEST_02에서 얻음
temp_responses = chain_2nd.batch(youtube_captions)
temp_responses

#### 6. full chain 생성 (첫번째 두번째 chain 합체)

In [15]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

full_chain = (
    {"query": RunnablePassthrough()} | 
    chain_1st |
    RunnableLambda(results_from_youtube) |
    chain_2nd.map()
)

In [17]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

full_chain = (
    {"query": RunnablePassthrough()} | 
    chain_1st |
    RunnableLambda(results_from_youtube) |
    {
        "youtube_results": RunnablePassthrough(),
        "chain_2nd_output": chain_2nd.map(),
    }
)

In [24]:
def flat_text(input_data):
    youtube_results = input_data['youtube_results']
    chain_2nd_output = input_data['summary_results']
    
    video_text = []
    
    for summary in chain_2nd_output:
        if not summary.segments:
            continue
        
        video_id = summary.video_id
        captions = next((item['captions'] for item in youtube_results if item['video_id'] == video_id), None)
        
        if captions:
            output = f"전체 요약 : {summary.description}\n\n본문 내용 :\n"
            
            for i, segment in enumerate(summary.segments, 1):
                start_time = segment.start
                end_time = start_time + segment.duration
                
                segment_text = ''
                for caption in captions:
                    if start_time <= caption['start'] < end_time or \
                       (caption['start'] < start_time and caption['start'] + caption['duration'] > start_time):
                        segment_text += caption['text'].replace('\n', ' ') + ' '
                
                output += f"{i}. {segment.title} : {segment_text.strip()}\n"
            
            video_text.append({"video_id": video_id, "text": output.strip()})
    
    return {"llm_query":youtube_results[0]['query'], "video_text":video_text}

In [25]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

full_chain = (
    {"query": RunnablePassthrough()} | 
    chain_1st |
    RunnableLambda(results_from_youtube) |
    {
        "youtube_results": RunnablePassthrough(),
        "summary_results": chain_2nd.map(),
    } |
    {
        "flat_text": RunnableLambda(flat_text),
        "summary_results": RunnablePassthrough(input_key="summary_results"),
    } |
    {
        "ranks": chain_3rd,
        "summary_results": RunnablePassthrough(input_key="summary_results"),
    }
) 


In [ ]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

full_chain = (
    {"query": RunnablePassthrough()} | 
    chain_1st |
    RunnableLambda(results_from_youtube) |
    {
        "youtube_results": RunnablePassthrough(),
        "chain_2nd_output": chain_2nd.map(),
    } |
    RunnableLambda(flat_text) |
    {
        "chain_2nd_output": RunnablePassthrough(),
        "chain_3rd_output": chain_3rd,
    }    
)

#### 7. full chain 실행

In [26]:
#input_query = """
#LF소나타 전조등 교체하는 법 알려줘
#"""
input_query = """
한국인 중에 기네스북에 오른 사람들은 누가 있으며 어떤 분야에서 올랐는지 알려줘
"""
responses = full_chain.invoke(input_query)

2024-07-18 23:02:22,643 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-18 23:02:22,660 - INFO - file_cache is only supported with oauth2client<4.0.0


youtube_query='한국인 기네스북 기록' llm_query='한국인 중에 기네스북에 오른 사람들과 그들이 어떤 분야에서 기록을 세웠는지 알려줘' language='ko'
{'channel': 'SBS Entertainment',
 'description': '스타킹 461회 20160809 SBS 아웃사이더는 기네스 기록에 도전하며 엄청난 속사포 랩을 선보인다. '
                '홈페이지 ...',
 'has_specified_caption': True,
 'id': 'IlGqIab-tRA',
 'publishedAt': '2016-08-09T13:11:16Z',
 'thumbnail': 'https://i.ytimg.com/vi/IlGqIab-tRA/default.jpg',
 'title': '아웃사이더, 기네스 세계 기록에 도전하는 엄청난 속사포 랩 도전! @스타킹 461회 20160809'}
{'channel': 'SBS 뉴스',
 'description': '브라질의 한 서퍼가 세계에서 가장 높은 파도를 타서 기네스 신기록을 세웠습니다. 그 장면 함께 보시죠. 원문 '
                '기사 더보기 ...',
 'has_specified_caption': True,
 'id': 'jx70ld3F7GA',
 'publishedAt': '2020-09-28T08:50:13Z',
 'thumbnail': 'https://i.ytimg.com/vi/jx70ld3F7GA/default.jpg',
 'title': '&#39;건물 6층&#39; 높이도 가뿐히…&#39;서핑 여제&#39; 신기록 세웠다 / SBS'}
{'channel': 'MBCNEWS',
 'description': "산타도, 트리도 아닙니다. 턱수염에 달린 장신구인데요. 미국에 사는 '조엘 스트라서'가 턱수염에 장신구를 가장 "
                '많이 올린 사람으로 ...',
 'has_specified_caption': True,
 'id': 

2024-07-18 23:02:30,503 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-18 23:02:30,816 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-18 23:02:31,392 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [27]:
responses

{'llm_query': '한국인 중에 기네스북에 오른 사람들과 그들이 어떤 분야에서 기록을 세웠는지 알려줘',
 'video_text': [{'video_id': 'IlGqIab-tRA',
   'text': '전체 요약 : 한국 대표가 기네스 기록에 도전하는 장면을 담았습니다.\n\n본문 내용 :\n1. 기네스 도전 소개 : - 뭔가 새로운 도전을 좀  해보도록 할 텐데요. 미국 래퍼 리키 브라운이 723절음을  51.27초에 했다고 하는데요. 한국 대표로 기네스 기록에 도전을  해보도록 하겠습니다.\n2. 도전 시작 : - 3 2 1 시작. 시작.\n3. 도전 성공 : - 우와. 50초 우와. - 와 뒷골 당겨. - 완전히 이겼어 완전히. 완전 이겼어요.'},
  {'video_id': 'jx70ld3F7GA',
   'text': '전체 요약 : 브라질 서퍼 마야 가베이라가 22.4m 파도를 타며 기네스 신기록을 세웠습니다.\n\n본문 내용 :\n1. 브라질 서퍼의 기네스 신기록 : -(캐스터) SNS와 인터넷 검색어를 통해 오늘 하루 관심사와 솔직한 반응을 알아보는 오클릭 시간입니다. 브라질의 한 서퍼가 세계에서 가장 높은 파도를 타서 기네스 신기록을 세웠습니다. 그 장면 함께 보시죠. 오클릭 첫 번째 검색어는 22m 파도 탄 서퍼입니다. 포르투갈 나자르 해안입니다. 거대한 파도가 밀려오는데요. 그 한가운데 서퍼 한 명이 서 있습니다. 파도는 점점 커지는데 서퍼는 오히려 빠른 속도의 파도를 즐기는 모습입니다. 이날 파도의 높이는 무려 22.4m로 건물 6층 정도의 높이였다고 하는데요. 이 서퍼는 바로 브라질의 서핑여제, 마야 가베이라였습니다. 이번 도전이 성공하면 가베이라는 기네스 신기록을 세웠는데요. 2018년 본인이 세웠던 세계기록 20.7m를 가뿐히 뛰어넘은 건 물론이고 최근 1년 동안 남녀 서퍼를 통틀어서 최고 기록을 세웠습니다. 7년 전 서핑 도중 다리가 부러져서 익사할 뻔까지 했던 가비에라는 사고 이후 서핑이 즐겁지 않았는데 그때 트라우마를

#### 8. 동영상 제작

In [ ]:
import uuid
job_id = str(uuid.uuid4())

- youtube 컨텐츠 download -> mp4 파일 생성

In [ ]:
from functions import download_youtube

for summary in responses:

    if not summary.segments:
        continue
    
    youtube_url = f"https://www.youtube.com/watch?v={summary.video_id}"

    file_path, file_name = download_youtube(youtube_url, job_id=job_id)
    print(f"Downloaded file names : {file_path}")

- 영상 slicing -> segment 영상 파일 생성

In [ ]:
from functions import slice_video

for summary in responses:

    content_id = job_id + "_" + summary.video_id

    segments = [(s.start, s.duration, content_id+"_"+str(i)+".mp4") for i, s in enumerate(summary.segments)]
    
    results = slice_video(content_id+".mp4", segments, height=540, width=960)


- YouTube 컨텐츠 별 설명 영상 생성
- segment별 설명 영상 생성, 영상에 text 삽입

In [ ]:
from functions import Position, draw_text, add_text_to_image, create_video_with_audio_and_image

segment_list = [] # concat 할 파일들을 순서대로 저장

for summary in responses:

    if not summary.segments:
        continue

    text_config = dict(
        position = Position.CENTER,
        font_color=(0, 128, 255),
        font_size=60,
        text = summary.description,
    )

    content_id = job_id + "_" + summary.video_id

    add_text_to_image("bg_image.png", content_id+".png", **text_config)

    text_config = dict(
        position = Position.TOP,
        font_color=(64, 64, 255),
        font_size=48,
        text = "채널 : "+summary.channel_name,
    )

    add_text_to_image(content_id+".png", content_id+".png", **text_config)

    create_video_with_audio_and_image(
        content_id+".png", 
        content_id+"_c.mp4", 
        text = summary.description,
        width = 640,
        height = 360,
    )

    segment_list.append(content_id+"_c.mp4")

    for i, s in enumerate(summary.segments):

        segment_id = content_id + "_" + str(i)

        text_config = dict(
            position = Position.CENTER,
            font_color=(0, 0, 255),
            font_size=80,
            text = s.title,
        )

        add_text_to_image("bg_image.png", segment_id+".png", **text_config)

        create_video_with_audio_and_image(
            segment_id+".png", 
            segment_id+"_c.mp4", 
            text = s.title,
            width = 640,
            height = 360,
            )
        
        segment_list.append(segment_id+"_c.mp4")

        text_config.update(
            position = Position.BOTTOM,
            color = (255, 0, 0),
            font_size = 72,
        )

        draw_text(segment_id+".mp4", [(0, s.duration, text_config)])

        segment_list.append(segment_id+"_t.mp4")


In [ ]:
# <UUID>_<video_id>_c.mp4 : video 표지 영상
# <UUID>_<video_id>_n_c.mp4 : n번 segment 표지 영상
# <UUID>_<video_id>_n_t.mp4 : n번 segment 영상

segment_list

- segment화된 영상들 이어 붙이기

In [ ]:
from functions import concatenate_videos

#output_file = "삼성SDS_FabriX.mp4"
output_file = "기네스북_한국인.mp4"

concatenate_videos(segment_list, output_file, width=640 , height=360 )